# Catcha Text Reading:

- <font size = '4'> There is a dataset of many captcha images where we need to **read** each captha image and **predict** the text in each one. 
    - The **actual text** is also given which is the **output** for the data set.
    - This training data set should be used to train the model.

- <font size = '4'> But before that we need to **extract the text** from each 
captcha image using **boundary boxes** segmentation

In [1]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import cv2
im = cv2.imread("/content/drive/MyDrive/Captcha Text Reading/captcha_18.png")
plt.imshow(im)
plt.show()

- <font size = '4'> This is the sample image of one captcha code in our data set.

In [ ]:
from google.colab.patches import cv2_imshow
im_1 = cv2.imread("/content/drive/MyDrive/Captcha Text Reading/captcha_18.png")
cv2_imshow(im_1)

In [ ]:
gray = cv2.cvtColor(im_1, cv2.COLOR_BGR2GRAY)

In [ ]:
gray

In [ ]:
cv2_imshow(gray)

- <font size = '4'> The colored image is been converted into gray scale image.

In [ ]:
blur = cv2.GaussianBlur(gray, (7,7), 0)

In [ ]:
cv2_imshow(blur)

In [ ]:
thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

In [ ]:
cv2_imshow(thresh)

- <font size = '4'> Done doing the threshold for the gray scale image
- <font size = '4'> Now need to find the **contours** for this thresold image.

In [ ]:
cnts = cv2.findContours(thresh, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)

In [ ]:
cnts = cnts[0] if len(cnts) == 2 else cnts[1]

In [ ]:
cnts = sorted(cnts, key = lambda x: cv2.boundingRect(x)[0])

In [ ]:
temp = 0
for c in cnts:
  x,y,w,h = cv2.boundingRect(c)
  cv2.rectangle(gray, (x,y), (x+w,y+h), (0,255,0), 1)
  temp = temp + 1
  if temp >= 5 and temp <= 9:
    letter = gray[y:y+h,x:x+w]
    cv2_imshow(letter)

In [ ]:
cv2_imshow(gray)

- <font size = '4'> Now lets do this bounding boxes for all captcha images at once and see which approach is better.


In [ ]:
import os
path = "/content/drive/MyDrive/Captcha Text Reading"
captcha_files = os.listdir(path)
captcha_files

In [ ]:
from pathlib import Path
path_names_tech = list(Path(path).glob('*.png'))
path_names_tech

In [ ]:
len(path_names_tech)

In [ ]:
len(captcha_files)

- <font size = '4'>Storing all the path and files in a list for further using.

In [ ]:
for i in captcha_files:
  im = cv2.imread("/content/drive/MyDrive/Captcha Text Reading/"+i)
  plt.imshow(im)
  plt.show()

In [ ]:
img_boxes = []
p =0
for i in captcha_files:
  p = p+1
  im = cv2.imread("/content/drive/MyDrive/Captcha Text Reading/"+i)
  gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
  blur = cv2.GaussianBlur(gray, (7,7), 0)
  thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
  cnts = cv2.findContours(thresh, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
  cnts = cnts[0] if len(cnts) == 2 else cnts[1]
  cnts = sorted(cnts, key = lambda x: cv2.boundingRect(x)[0])
  temp = 0
  w_lst = []
  for c in cnts:
    temp = temp + 1
    x,y,w,h = cv2.boundingRect(c)
    cv2.rectangle(gray, (x,y), (x+w,y+h), (0,255,0), 1)
    if temp >= 5 and temp <= 9:
        w_lst.append(w)
  print(w_lst)
  
  cv2_imshow(gray)

In [ ]:
print(p)

- <font size = '4'> We can see this approach is **better** when compared with the first approach. 
- So, lets **finalize** this approcach to plot the bounding boxes for each character.

- <font size = '4'> But, here the only problem is if 2 or 3 cahacrters touch to each other without any space between them then only one bounding box is been plotted for those 2 or 3 characters mixed.

- <font size = '4'> For this one solution is,

##**SOLUTION**


- <font size = '4'> Finding the approx range of length of a single character bounding box and if the bounding boxes of other exceeds this range, that means that the box includes more than 1 character.
- So, this box should be split into parts so that the lenght of each box gets into the above range of single character bounding box length.

- <font size = '4'> First lets try for a single image which is captcha_18

In [ ]:
im_1 = cv2.imread("/content/drive/MyDrive/Captcha Text Reading/captcha_18.png")
cv2_imshow(im_1)

In [ ]:
gray = cv2.cvtColor(im_1, cv2.COLOR_BGR2GRAY)

- <font size = '4'> The colored image is been converted into gray scale image.

In [ ]:
blur = cv2.GaussianBlur(gray, (7,7), 0)

In [ ]:
cv2_imshow(blur)

In [ ]:
thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

In [ ]:
cv2_imshow(thresh)

- <font size = '4'> Done doing the threshold for the gray scale image
- <font size = '4'> Now need to find the **contours** for this thresold image.

In [ ]:
cnts = cv2.findContours(thresh, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)

In [ ]:
cnts = cnts[0] if len(cnts) == 2 else cnts[1]

In [ ]:
cnts = sorted(cnts, key = lambda x: cv2.boundingRect(x)[0])

In [ ]:
temp = 0
for c in cnts:
  temp = temp+1
  x,y,w,h = cv2.boundingRect(c)
  cv2.rectangle(gray, (x,y), (x+w,y+h), (0,255,0), 1)
  #print(w)
  if temp >= 5 and temp <= 9:
    if w >= 25 and w < 39:
      cv2.rectangle(gray, (x,y), (x+int(w/2),y+h), (0,255,0), 1)
      cv2.rectangle(gray, (x+int(w/2),y), (x+w,y+h), (0,255,0), 1)
      letter = gray[y:y+h,x:x+int(w/2)]
      cv2_imshow(letter)
      letter = gray[y:y+h,x+int(w/2):x+w]
      cv2_imshow(letter)
    else:
      letter = gray[y:y+h,x:x+w]
      cv2_imshow(letter)
    

In [ ]:
cv2_imshow(gray)

- <font size = '4'> Successfully spilted the 2 character boundary box into seperate boxes

- <font size = '4'> Lets also check for the case if  3 charcters are together having one boundary box.

In [ ]:
im_1 = cv2.imread("/content/drive/MyDrive/Captcha Text Reading/captcha_14.png")
cv2_imshow(im_1)

In [ ]:
gray = cv2.cvtColor(im_1, cv2.COLOR_BGR2GRAY)

- <font size = '4'> The colored image is been converted into gray scale image.

In [ ]:
cv2_imshow(gray)

In [ ]:
blur = cv2.GaussianBlur(gray, (7,7), 0)

In [ ]:
cv2_imshow(blur)

In [ ]:
thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

In [ ]:
cv2_imshow(thresh)

- <font size = '4'> Done doing the threshold for the gray scale image
- <font size = '4'> Now need to find the **contours** for this thresold image.

In [ ]:
cnts = cv2.findContours(thresh, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)

In [ ]:
cnts = cnts[0] if len(cnts) == 2 else cnts[1]

In [ ]:
cnts = sorted(cnts, key = lambda x: cv2.boundingRect(x)[0])

In [ ]:
for c in cnts:
  x,y,w,h = cv2.boundingRect(c)
  cv2.rectangle(gray, (x,y), (x+w,y+h), (0,255,0), 1)
  print(w)

In [ ]:
cv2_imshow(gray)

- <font size = '4'>Here **W,Q,2** are having one boundary box. Lets split this box into 3 boxes.


In [ ]:
temp = 0
for c in cnts:
  temp = temp+1
  x,y,w,h = cv2.boundingRect(c)
  cv2.rectangle(gray, (x,y), (x+w,y+h), (0,255,0), 1)
  #print(w)
  if temp >= 5 and temp <= 9:
    if w >= 39:
      cv2.rectangle(gray, (x,y), (x+round(w/3),y+h), (0,255,0), 1)
      cv2.rectangle(gray, (x+round(w/3),y), (x+round((2*w)/3),y+h), (0,255,0), 1)
      cv2.rectangle(gray, (x+round((2*w)/3),y), (x+w,y+h), (0,255,0), 1)
      
    

In [ ]:
cv2_imshow(gray)

- <font size = '4'> Successfully converted the big box into 3 boxes.

- <font size = '4'> Now lets aply this approach for all the captcha images which are above.

In [ ]:
img_boxes = []
for i in captcha_files:
  im = cv2.imread("/content/drive/MyDrive/Captcha Text Reading/"+i)
  gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
  blur = cv2.GaussianBlur(gray, (7,7), 0)
  thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
  cnts = cv2.findContours(thresh, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
  cnts = cnts[0] if len(cnts) == 2 else cnts[1]
  cnts = sorted(cnts, key = lambda x: cv2.boundingRect(x)[0])
  temp = 0
  for c in cnts:
    temp = temp+1
    x,y,w,h = cv2.boundingRect(c)
    cv2.rectangle(gray, (x,y), (x+w,y+h), (0,255,0), 1)
    #print(w)
    if temp >= 5 and temp <= 9:
      if w >= 25 and w < 39:
        cv2.rectangle(gray, (x,y), (x+int(w/2),y+h), (0,255,0), 1)
        cv2.rectangle(gray, (x+int(w/2),y), (x+w,y+h), (0,255,0), 1)
      elif w >= 39:
        cv2.rectangle(gray, (x,y), (x+round(w/3),y+h), (0,255,0), 1)
        cv2.rectangle(gray, (x+round(w/3),y), (x+round((2*w)/3),y+h), (0,255,0), 1)
        cv2.rectangle(gray, (x+round((2*w)/3),y), (x+w,y+h), (0,255,0), 1)
      
  cv2_imshow(gray)

##**TRAINING**

- <font size = '4'> Finally these are the best bounding boxes plotting and which can be used to further training.

- <font size = '4'> For training purpose lets take the images that contains single character bounding boxes instead of 2or3 chracters merged boxes for better model traning.

In [ ]:
single_char_captcha_files = []
p =0
for i in captcha_files:
  im = cv2.imread("/content/drive/MyDrive/Captcha Text Reading/"+i)
  gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
  blur = cv2.GaussianBlur(gray, (7,7), 0)
  thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
  cnts = cv2.findContours(thresh, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
  cnts = cnts[0] if len(cnts) == 2 else cnts[1]
  cnts = sorted(cnts, key = lambda x: cv2.boundingRect(x)[0])
  temp = 0
  w_lst = []
  for c in cnts:
    temp = temp + 1
    x,y,w,h = cv2.boundingRect(c)
    cv2.rectangle(gray, (x,y), (x+w,y+h), (0,255,0), 1)
    if temp >= 5 and temp <= 9:
        w_lst.append(w)
  for j in w_lst:
    if j>=25:
      break
  else:
    p =p + 1
    single_char_captcha_files.append(i)
    cv2_imshow(gray)

In [ ]:
print(p)

- <font size = '4'> These 145 captcha images are the ones which are having bounding boxes only for 1 character.

In [ ]:
single_char_captcha_files

In [ ]:
print(len(single_char_captcha_files))

These are the files names of the captcha images that contains only one character in each bounding boxes.

- <font size = '4'> Now lets try to create a data frame with the individual character boxes of these 145 captcha image.

In [2]:
lst_char_boxes = []
z = -1
remove = []
for i in single_char_captcha_files:
  z = z+1
  im = cv2.imread("/content/drive/MyDrive/Captcha Text Reading/"+i)
  gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
  blur = cv2.GaussianBlur(gray, (7,7), 0)
  thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
  cnts = cv2.findContours(thresh, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
  cnts = cnts[0] if len(cnts) == 2 else cnts[1]
  cnts = sorted(cnts, key = lambda x: cv2.boundingRect(x)[0])
  temp = 0
  lst_img_wise_char_boxes = []
  for c in cnts:
    temp = temp+1
    x,y,w,h = cv2.boundingRect(c)
    cv2.rectangle(gray, (x,y), (x+w,y+h), (0,255,0), 1)
    if temp >= 5 and temp <= 10:
      letter = gray[y:y+h,x:x+w]
      cv2_imshow(letter)
      letter = letter.tolist()
      lst_img_wise_char_boxes.append(letter)
  if len(lst_img_wise_char_boxes) == 6:
    for q in lst_img_wise_char_boxes:
      lst_char_boxes.append(q)      
    cv2_imshow(gray)
  else:
    remove.append(z)

NameError: name 'single_char_captcha_files' is not defined

In [ ]:
letter

In [ ]:
lst_char_boxes

In [ ]:
"""
length_all_letters = []
for i in lst_char_boxes:
  length = max(map(len, i))
  length_all_letters.append(length)
max_length = max(length_all_letters)
for i in lst_char_boxes:
  for r in i:
    for j in range(max_length-len(r)):
      r.append(255)

"""

In [ ]:
remove

In [ ]:
type(lst_char_boxes)

In [ ]:
for i in remove:
  single_char_captcha_files.pop(i)

In [ ]:
len(single_char_captcha_files)

- <font size = '4'> Now, this is a 3d array containing pixels valued lists of 846 letters.

- <font size = '4'> Lets also create a list of the target variables.

In [ ]:
import pandas as pd

dict_captcha_text = pd.read_csv('/content/drive/MyDrive/captcha_verification/captcha_verification.csv', header=None, index_col=0, squeeze=True).to_dict()
print(dict_captcha_text)

In [ ]:
dict_captcha_text.pop('FileName')
dict_captcha_text

In [ ]:
lst_text = []
for i in single_char_captcha_files:
  lst_img_wise_text = list(dict_captcha_text[i])
  lst_text.append(lst_img_wise_text)
  

In [ ]:
lst_text

In [ ]:
lst_text = sum(lst_text, [])
print(lst_text)

In [ ]:
len(lst_text)

- <font size = '4'> So, finally the lists required for training are,
           1.   lst_char_boxes
           2.   lst_text

##  DATA PREPARATION:

In [ ]:
train_images = lst_char_boxes

In [ ]:
train_labels = lst_text

In [ ]:
train_labels

- <font size = '4'> The complete 846 elements are been taken in this train dataset.
- Lets split the dataset for validation set also.

In [ ]:
from sklearn.model_selection import train_test_split

train_images,val_images,train_labels,val_labels = train_test_split(
                                               train_images,train_labels,test_size = 1/6,shuffle = True,random_state = 999999)

- <font size = '4'> Done spliting the data set into training and validation sets
- <font size = '4'> But these datasets are of list type we need to convert them into **numpy** arrays for training purpose. 

In [ ]:
import numpy as np

train_images = np.array([np.array(xi) for xi in train_images])

train_labels = np.array([np.array(xl) for xl in train_labels])

val_images = np.array([np.array(xi) for xi in val_images])

val_labels = np.array([np.array(xl) for xl in val_labels])


In [ ]:
type(train_images),type(train_labels),type(val_images),type(val_labels)

In [ ]:
train_images.shape

In [ ]:
train_labels.shape

In [ ]:
val_images.shape

In [ ]:
val_labels.shape

- <font size = '4'> Now the data set is ready for training.

In [ ]:
plt.figure()
plt.imshow(train_images[0])
plt.colorbar()
plt.grid(False)
plt.show()

- <font size = '4'> The pixel scale is 0-255 as seen in the figure lets convert the scale to 0-1 for better training purpose.

In [ ]:
train_images = train_images/255

In [ ]:
val_images = val_images/255

In [ ]:
plt.figure()
plt.imshow(train_images[0])
plt.colorbar()
plt.grid(False)
plt.show()

In [ ]:
train_labels[0]

In [3]:
X_train = train_images
y_train = train_labels
X_val = val_images
y_val = val_labels

NameError: name 'train_images' is not defined

In [ ]:
X_train

## SETTING UP LAYERS

In [ ]:
import tensorflow as tf
model = tf.keras.Sequential([
   tf.keras.layers.Flatten(input_shape = (15,15)),
   tf.keras.layers.Dense(128, activation = 'relu'),
   tf.keras.layers.Dense(62) 
])

- <font size = '4'> First layer in this model, **Flatten** tranforms the 2-d array into a single dimension array contains 28*28 = 784 units.
- <font size = '4'> Next we created another layer which is **Dense layer** creating 128 nodes where each node is linked with all the 784 units.
-  <font size = '4'> Now lets create the output layer, **Dense layer** of 10 nodes as our **output feature** contains 10 classes. <br>  
    -   Here also each node is connected with the above layer's 784 nodes and here each node contains **score** that indicates the current image belongs to one of the 10 classes.

In [ ]:
model.compile(optimizer = 'adam',
             loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),
              metrics = ['accuracy'])

##TRAINGING

-<font size = '4'> The list of a character image should be in a 2 dimension and with equal dimensions for each element in the array.<br>
- <font size = '4'>But here the dimension for each character is different size.<br>
<font size = '7'> **WHY**

- <font size = '4'> We can see the dimensions of each image are **different** with each other. <br>
- So, the flatteing of the dataset is not happening.

In [ ]:
for i in range(len(X_train)):
  print(X_train[i].shape)

In [ ]:
for i in range(len(X_train)):
  dim = (15, 15)
  X_train[i] = cv2.resize(np.float32(X_train[i]), dim)

In [ ]:
for i in range(len(X_val)):
  dim = (15, 15)
  X_val[i] = cv2.resize(np.float32(X_val[i]), dim)

In [ ]:
for i in range(len(X_train)):
  print(X_train[i].shape)

In [ ]:
for i in range(len(X_train)):
  X_train[i] = X_train[i].tolist()

In [ ]:
X_train = X_train.tolist()

In [ ]:
for i in range(len(X_val)):
  X_val[i] = X_val[i].tolist()

In [ ]:
X_val = X_val.tolist()

In [ ]:
for i in range(len(y_train)):
  y_train[i] = y_train[i].tolist()

In [ ]:
y_train = y_train.tolist()

In [ ]:
for i in range(len(y_val)):
  y_val[i] = y_val[i].tolist()

In [ ]:
y_val = y_val.tolist()

In [4]:
y_train

NameError: name 'y_train' is not defined

In [ ]:
y_val


- <font size = '4'> Done standardizing of all the images in the dataset.
<br>
- Now, lets try training

In [ ]:
X_train =np.array([np.array(xi) for xi in X_train])


In [ ]:
X_val =np.array([np.array(xv) for xv in X_val])


In [ ]:
X_train

In [ ]:
y_train =np.array([np.array(yi) for yi in y_train])


In [ ]:
y_val =np.array([np.array(yi) for yi in y_val])


In [ ]:
y_train

In [ ]:
X_train.shape

In [ ]:
plt.figure()
plt.imshow(X_train[1])
plt.colorbar()
plt.grid(False)
plt.show()

In [ ]:
y_train[1]

In [ ]:
model.fit(X_train, y_train, epochs = 10)